In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  train_test_split
from sklearn.decomposition import PCA
import tldextract
import re

In [14]:
df = pd.read_csv(r'C:/Users/eprmitr/Desktop/Blogs/AI-ML/Malicious_urls.csv', sep = ',')

In [15]:
print("The sample size of the dataset is:",df.shape[0],"rows and ", df.shape[1], "features")

The sample size of the dataset is: 420464 rows and  2 features


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420464 entries, 0 to 420463
Data columns (total 2 columns):
url      420464 non-null object
label    420464 non-null object
dtypes: object(2)
memory usage: 6.4+ MB


In [17]:
df.head(5)

url label
0  diaryofagameaddict.com   bad
1        espdesign.com.au   bad
2      iamagameaddict.com   bad
3           kalantzis.net   bad
4   slightlyoffcenter.net   bad

In [18]:
#!pip install tldextract

In [19]:
def parsing(url):
    subdomain, domain, domain_suffix = (i for i in tldextract.extract(url))
    return [subdomain, domain, domain_suffix]

def ETL(data):
    data2 = data['url']
    #print(data2)
    data3 = [parsing(i) for i in data2]
    #print(data3)
    data3 = pd.DataFrame(data3,columns=['subdomain', 'domain', 'TLD'])
    data = data.reset_index(drop=True)
    data = pd.concat([data,data3], axis=1)
    return data

In [20]:
df=ETL(df)

In [21]:
df.head()

url label subdomain              domain     TLD
0  diaryofagameaddict.com   bad            diaryofagameaddict     com
1        espdesign.com.au   bad                     espdesign  com.au
2      iamagameaddict.com   bad                iamagameaddict     com
3           kalantzis.net   bad                     kalantzis     net
4   slightlyoffcenter.net   bad             slightlyoffcenter     net

In [22]:
df.isnull().sum()

url          0
label        0
subdomain    0
domain       0
TLD          0
dtype: int64

In [23]:
df.label.value_counts().plot(kind='bar')

# Data Preparation

We dont need the "URL" anymore so lets drop it

In [24]:
df=df.drop(['url'],axis=1)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420464 entries, 0 to 420463
Data columns (total 4 columns):
label        420464 non-null object
subdomain    420464 non-null object
domain       420464 non-null object
TLD          420464 non-null object
dtypes: object(4)
memory usage: 12.8+ MB


In [33]:
df[['subdomain','domain','TLD']]

subdomain                     domain     TLD
0                            diaryofagameaddict     com
1                                     espdesign  com.au
2                                iamagameaddict     com
3                                     kalantzis     net
4                             slightlyoffcenter     net
5                                  toddscarwash     com
6                                    tubemoviez     com
7                                           ipl      hk
8                                   crackspider      us
9                                    pos-kupang     com
10                                        rupor    info
11                               svision-online      de
12          officeon                         ch      ma
13                                      sn-gzzx     com
14                                       sunlux     net
15                                      outporn     com
16      timothycopus                      aimoo     com
17                                  xindalawyer     com
18       freeserials                        spb      ru
19                         deletespyware-adware     com
20         orbowlada                     strefa      pl
21                                    ruiyangcn     com
22                                         zkic     com
23         adserving            favorit-network     com
24                                       cracks      vg
25                              juicypussyclips     com
26                                nuptialimages     com
27                                    andysgame     com
28                                  bezproudoff      cz
29                               ceskarepublika     net
...              ...                        ...     ...
420434           www                   alfalima      it
420435                            rapiseebrains     com
420436                                 fuji-ncb  com.pk
420437                                  aadroid     net
420438           mit             fileserver4390     org
420439                           cureeczemafast     org
420440                          glutenfreeworks     com
420441                                dataplues     com
420442                             60.250.76.52        
420443                                   f4321y     com
420444                                  mykings      pw
420445                       activatemywebsetup     com
420446           www              iemailpremium     com
420447                       highpowerresources     com
420448           new              ASKGRANNYSHOP     COM
420449          find  burnsmarketingandresearch     com
420450           gfd           DATINGUPPERCLASS     COM
420451           rty      freebiesfortheover60s     com
420452                                  defibel     org
420453                           stefanocardone     com
420454                                  defibel     org
420455                          shapingsoftware     com
420456          free                    ulohapp    info
420457          free                    ulohapp    info
420458           mol                     com-ho      me
420459                           23.227.196.215        
420460                            apple-checker     org
420461                             apple-iclods     org
420462                            apple-uptoday     org
420463                             apple-search    info

[420464 rows x 3 columns]

In [37]:
df.head()

label subdomain              domain     TLD
0   bad            diaryofagameaddict     com
1   bad                     espdesign  com.au
2   bad                iamagameaddict     com
3   bad                     kalantzis     net
4   bad             slightlyoffcenter     net

All the features are categorical so lets encode all of them

In [38]:
le = LabelEncoder()
col = df.columns
df[col] = df[col].apply(lambda c:le.fit_transform(c))

# Model Train & Test

In [41]:
Y=df.label
X=df.drop(['label'],axis=1)
### splitting data in training and test set
x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=0)

### Shape of the training and test set
print("====x_train,x_test,y_train,y_test shapes are as below====")
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

====x_train,x_test,y_train,y_test shapes are as below====
(336371, 3) (84093, 3) (336371,) (84093,)


# Random Forest Classification

In [45]:
%%time
rforest=RandomForestClassifier(criterion='entropy')
rforest.fit(x_train,y_train)
y_pred=rforest.predict(x_test)
print("====Confusion Matrix====")
print("\n")
cnf_matrix_rforest=confusion_matrix(y_test,y_pred)
print(cnf_matrix_rforest)
print("\n")
print("======Model Accuracy=======") 
print ("DT Train Accuracy :: ", accuracy_score(y_train, dtree.predict(x_train)))
print ("DT Test Accuracy  :: ", accuracy_score(y_test, y_pred))
print("\n")
print("====Classification Report====")
print("\n")
print(classification_report(y_test,y_pred))

C:\Users\eprmitr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


====Confusion Matrix====


[[13135  2039]
 [ 2551 66368]]


======Model Accuracy=======
DT Train Accuracy ::  0.9992508272116205
DT Test Accuracy  ::  0.9454175734008776


====Classification Report====


              precision    recall  f1-score   support

           0       0.84      0.87      0.85     15174
           1       0.97      0.96      0.97     68919

   micro avg       0.95      0.95      0.95     84093
   macro avg       0.90      0.91      0.91     84093
weighted avg       0.95      0.95      0.95     84093

Wall time: 7.79 s


# Decision Tree Classification

In [55]:
%%time
dtree=DecisionTreeClassifier(max_depth=100,criterion='entropy')
dtree.fit(x_train,y_train)
y_pred=dtree.predict(x_test)
cnf_matrix_dtree=confusion_matrix(y_test,y_pred)
print("======Confusion Matrix=======") 
print(cnf_matrix_dtree)
print('\n')
print("======Model Accuracy=======") 
print ("DT Train Accuracy :: ", accuracy_score(y_train, dtree.predict(x_train)))
print ("DT Test Accuracy  :: ", accuracy_score(y_test, y_pred))
print('\n')
print("======Classification Report=======") 
print(classification_report(y_test,y_pred))

======Confusion Matrix=======
[[13163  2011]
 [ 2375 66544]]


======Model Accuracy=======
DT Train Accuracy ::  0.9992508272116205
DT Test Accuracy  ::  0.9478434590275052


======Classification Report=======
              precision    recall  f1-score   support

           0       0.85      0.87      0.86     15174
           1       0.97      0.97      0.97     68919

   micro avg       0.95      0.95      0.95     84093
   macro avg       0.91      0.92      0.91     84093
weighted avg       0.95      0.95      0.95     84093

Wall time: 1.63 s
